<a href="https://colab.research.google.com/github/datascience-uniandes/data-quality-tutorial/blob/master/data-quality-tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Quality and Cleanliness

MINE-4101: Applied Data Science  
Univerisdad de los Andes  
  
**Dataset:** Homicides Colombia ([datos.gov.co](datos.gov.co))

**References:**
- String similarity metrics: https://www.baeldung.com/cs/string-similarity-edit-distance
- Levenshtein lib's docs: https://rapidfuzz.github.io/Levenshtein/installation.html
  
Last update: September, 2024

In [ ]:
!pip install levenshtein

In [ ]:
import re
from datetime import datetime

import numpy as np
import pandas as pd

from Levenshtein import distance, ratio

In [ ]:
pd.set_option("display.max_columns", None)

### Load the data

In [82]:
dataset_url="https://github.com/amrios80/data-quality-tutorial/raw/master/data/homicides.csv"
homicides_df = pd.read_csv(dataset_url)

In [83]:
homicides_df.shape

(12400, 22)

In [84]:
homicides_df.dtypes

,0
FECHA,object
DEPARTAMENTO,object
MUNICIPIO,object
DIA,object
HORA,object
BARRIO,object
ZONA,object
CLASE DE SITIO,object
ARMA O MEDIO,object
MOVIL VICTIMA,object


In [85]:
homicides_df.sample(5)

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,HORA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
11321,12/05/2021 12:00:00 AM,HUILA,LA PLATA,Sábado,8:40,VEREDA BELEN,RURAL,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,49.0,FEMENINO,SOLTERO,AMA DE CASA,NO REPORTADO,PRIMARIA,COLOMBIA,41396000.0,1966.0,72-031,hilgkh9352@unidatos.edu.co
6668,07/15/2021 12:00:00 AM,VALLE,CALI (CT),Miércoles,19:30,ALFONSO B. ARAGON E14,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,68.0,MASCULINO,UNION LIBRE,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,76001000.0,1947.0,18-293,cublsl3223@gmail.com
3880,04/27/2021 12:00:00 AM,N. DE SANTANDER,CÚCUTA (CT),Lunes,20:20,BARRIO SAN MIGUEL,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,38.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,54001000.0,1977.0,61-259,bdgfcl2676@unidatos.edu.co
4286,05/10/2021 12:00:00 AM,CUNDINAMARCA,BOGOTÁ D.C. (CT),Domingo,9:30,CHUNIZA ENO REPORTADO5,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,27.0,MASCULINO,UNION LIBRE,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,11001000.0,1988.0,24-676,umauho6276@gmail.com
2104,03/06/2021 12:00:00 AM,CAQUETÁ,FLORENCIA (CT),Voernes,8:40,BERLIN,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,VEHICULO,NO REPORTADO,34.0,MASCULINO,UNION LIBRE,COMERCIANTE,NO REPORTADO,SECUNDARIA,COLOMBIA,18001000.0,1981.0,82-222,bbbatc7760@unidatos.edu.co


### Work with datetimes

In [ ]:
# Creating a lambda expression for datetime parsing
dateparse = lambda x: datetime.strptime(x, "%m/%d/%Y %H:%M:%S %p")

In [ ]:
# Applying the validation to all values in the column
homicides_df["FECHA"].apply(dateparse)

# IT IS EXPECTED TO HAVE AN ERROR BECAUSE SOME VALUES DOESN'T FIT THE FORMAT

In [ ]:
# Creating a function for validating which value is causing the previous error
def error_in_format(x):
    try:
        datetime.strptime(x, "%m/%d/%Y %H:%M:%S %p")
        return False
    except:
        return True

In [ ]:
# Using the function for validation
homicides_df.loc[homicides_df["FECHA"].apply(error_in_format)]

In [ ]:
# Deleting a row by its index
homicides_df.drop([486, 695, 1250, 12168, 12399], inplace=True)

In [ ]:
# Trying to parse the datetime string again
homicides_df["FECHA"] = homicides_df["FECHA"].apply(dateparse)

In [ ]:
homicides_df.dtypes

In [ ]:
# Counting homicides by hour
homicides_df["FECHA"].dt.hour.value_counts()

# All datetime hour parts are the same

<span style="color:red">The homicide hour is available in a different column!</span>

In [ ]:
# Merging both columns
homicides_df["FECHA"] = homicides_df["FECHA"].astype(str).apply(lambda x: x[:11]) + homicides_df["HORA"]

In [ ]:
# Deleting redundant column
homicides_df.drop(columns=["HORA"], inplace=True)

In [ ]:
# Making a new expression for datetime parsing
dateparse = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M")

In [ ]:
# Applying the expression
homicides_df["FECHA"] = homicides_df["FECHA"].apply(dateparse)

In [ ]:
homicides_df.dtypes

In [ ]:
homicides_df.sample(5)

In [ ]:
homicides_df["FECHA"].min(), homicides_df["FECHA"].max()

### Fix categorical column

In [ ]:
# Creating a dictionary representing the valid departments for Colombia
departments_list = ['ANTIOQUIA', 'ATLÁNTICO', 'BOLÍVAR', 'BOYACÁ', 'CALDAS', 'CAQUETÁ',
       'CASANARE', 'CAUCA', 'CESAR', 'CHOCÓ', 'CÓRDOBA', 'META',
       'CUNDINAMARCA', 'HUILA', 'MAGDALENA', 'NARIÑO', 'PUTUMAYO',
       'RISARALDA', 'SANTANDER', 'SUCRE', 'TOLIMA', 'VALLE',
       'NORTE DE SANTANDER', 'GUAJIRA', 'QUINDÍO', 'SAN ANDRÉS Y PROVIDENCIA', 'ARAUCA',
       'GUAINÍA', 'VICHADA', 'VAUPÉS', 'GUAVIARE', 'AMAZONAS']

In [ ]:
# Finding values not matching with the dictionary
homicides_df.loc[~homicides_df["DEPARTAMENTO"].isin(departments_list), "DEPARTAMENTO"].unique()

<span style="color:red">TODO: Replace the values identified as error to a valid value from the dictionary.</span>

<span style="color:red">Hint: You can use the replace() pandas function.</span>

### Analyze potential duplicates

In [ ]:
# Detecting duplicates by "CÉDULA" column
duplicates_by_cedula = homicides_df.loc[homicides_df["CÉDULA"].duplicated(keep=False)]

In [ ]:
duplicates_by_cedula.shape

In [ ]:
duplicates_by_cedula.sort_values("CÉDULA", ascending=True).head(6)

<span style="color:red">TODO: Delete records with "CÉDULA" duplicated.</span>

<span style="color:red">Hint: You can use drop_duplicates() pandas function.</span>

### Fix formats

In [ ]:
# Using regular expressions for validating if "CÉDULA" values match the pattern XX-XXX
cedula_malformed = homicides_df.loc[homicides_df["CÉDULA"].apply(lambda x: (re.match("\d{2}-\d{3}", x) is None))]

In [ ]:
cedula_malformed.shape

In [ ]:
cedula_malformed.head(6)

<span style="color:red">TODO: Fix the malformed "CÉDULA" values.</span>

<span style="color:red">Assume that the third digit in the first part corresponds to the first one in the second part.</span>

<span style="color:red">TODO:Check the "CORREO" column (PATTERN: 4 digits before the @, only .edu.co and .com domains are allowed)</span>

### Automate imputation of categorical values

In [ ]:
homicides_df["DIA"].unique()

In [ ]:
# Calculating the distance between two words using the Levenshtein distance
print(distance("sábado", "sabaod"))
print(distance("sábado", "viernes"))

In [ ]:
print(ratio("sábado", "sabaod"))
print(ratio("sábado", "viernes"))

<span style="color:red">TODO: Create a function to automatically fix the digitation errors for the "DIA" column.</span>

In [ ]:
# It's expected to hava a list with only 7 values
homicides_df["DIA"].unique()

### Recalculate based on a different column

In [ ]:
homicides_df[["AÑO DE NACIMIENTO", "EDAD"]].sample(10)

<span style="color:red">TODO: Fix "EDAD" column using "AÑO DE NACIMIENTO".</span>

### Conclusion

<span style="color:red">Make a summary of the different data quality issues found on the dataset, the data quality dimension that is related to and the implemented strategy for solving or mitigating each issue.</span>